spo

In [244]:

# 접속테스트
try:
    db = manager.db_connect_and_return()
    print('접속 성공')
except Exception as e:
    print(f'접속문제 발생 : {e}')

접속 성공


In [242]:
# 데이터 읽어봐서 표시
query_search = f" SELECT * FROM {_TB_NAME}"

# query_insert = f" INSERT INTO {_TB_NAME} (Index, site, 전체공정, 세부공정, 유형, Cost, month) VALUES ({}, {}, {}, {}, {}, {}, {});"

rst = manager.search(query_search)

import pandas as pd

df = pd.DataFrame(rst)
df

,Index,site,전체공정,세부공정,유형,Cost,month
0,27272,오창,조립,Notching,전공정 Loss,0.000000,1월
1,27273,오창,조립,Lamination,전공정 Loss,0.211953,1월
2,27274,오창,조립,Stacking & Folding,전공정 Loss,0.152065,1월
3,27275,오창,조립,Packaging,전공정 Loss,0.099265,1월
4,27276,오창,조립,Lamination,쇼트 불량,0.594672,1월
...,...,...,...,...,...,...,...
4230,31502,ESNB,활성화,특성측정,Taping(%),0.149945,7월
4231,31503,ESNB,활성화,특성측정,Sealing 주름(%),0.000000,7월
4232,31504,ESNB,활성화,특성측정,Cold IR(%),0.000000,7월
4233,31505,ESNB,활성화,특성측정,절연 저항(%),0.983421,7월


In [1]:

# 테스트용 데이터 생성
df = pd.read_csv('output/전체공정.csv')
b = df.iloc[3,:]


NameError: name 'pd' is not defined

 INSERT INTO csv.loss_list (`site`, `전체공정`, `세부공정`, `유형`, `Cost`, `month`) VALUES ('오창', '활성화', 'Formation1', '그리퍼접촉(%)', '0.13476578889526009', '1월');
 INSERT INTO csv.loss_list (`site`, `전체공정`, `세부공정`, `유형`, `Cost`, `month`) VALUES ('오창', '활성화', 'Formation1', '충전전압(%)', '0.116851667763995', '1월');
 INSERT INTO csv.loss_list (`site`, `전체공정`, `세부공정`, `유형`, `Cost`, `month`) VALUES ('오창', '활성화', 'Formation1', '(Jig)가압단락 검사(%)', '0.16411566665382596', '1월');
 INSERT INTO csv.loss_list (`site`, `전체공정`, `세부공정`, `유형`, `Cost`, `month`) VALUES ('오창', '활성화', 'Formation1', '외관변형 작업불가(%)', '0.09575575530839472', '1월');
 INSERT INTO csv.loss_list (`site`, `전체공정`, `세부공정`, `유형`, `Cost`, `month`) VALUES ('오창', '활성화', 'Formation1', 'J/F 온도(%)', '0.0', '1월');
 INSERT INTO csv.loss_list (`site`, `전체공정`, `세부공정`, `유형`, `Cost`, `month`) VALUES ('오창', '활성화', 'Formation1', 'J/F 용량(%)', '0.0', '1월');
 INSERT INTO csv.loss_list (`site`, `전체공정`, `세부공정`, `유형`, `Cost`, `month`) VALUES ('오창', '활성화', 'Formation1', 

UPDATE 용 코드 / INSERT를 하는 경우 이어쓰므로 중복데이터가 발생하므로 업데이트 체크하여 같은 날짜에 중복데이터가 발생하지 않도록 함.

!! 현재 일자별로 되어있는데 월별로 바꿔야함

1. 업데이트 날짜 확인
2. 날짜가 오늘 이전이면 추가데이터 삽입(이것도 전체데이터로 되어있어서 수정필요)
3. 이미 업데이트되었으면 종료

In [ ]:

# 중복으로 INSERT 되지 않도록 일자별로 관리. --> 오늘날짜로 업데이트되었으면 INSERT 코드 실행하지 않음.
from datetime import datetime
row = manager.search('SELECT update_time from update_check')

today = datetime.today().date()

if row[0]['update_time'] != datetime.today().date():
    _TB_NAME = 'update_check'
    query_update = f"UPDATE {_DB_NAME}.{_TB_NAME} SET update_time = '{str(today)}' where table_name like 'loss_list'"
    manager.update(query_update)


    _TB_NAME = 'loss_list'
    for i in range(len(df)):
        row = df.iloc[i,:]
        query_insert = f" INSERT INTO {_DB_NAME}.{_TB_NAME} (`site`, `전체공정`, `세부공정`, `유형`, `Cost`, `month`) " \
                       f"VALUES ('{row[0]}', '{row[1]}', '{row[2]}', '{row[3]}', '{row[4]}', '{row[5]}');"

        manager.insert(query_insert)
    manager.commit()

else:
    print('이미 최신 자료입니다.')





전체삭제 query임


In [ ]:
# 테스트용 삭제 쿼리

query_delete = f"DELETE FROM {_TB_NAME} WHERE {_TB_NAME}.전체공정 = '조립' or 전체공정 = '활성화' or 전체공정 = '전극' ;"
manager.delete(query_delete)

# 테스트용으로 만든 임시코드
from datetime import datetime

query_update = f"UPDATE {_DB_NAME}.{'update_check'} SET update_time = '{str(today)}' where table_name like 'loss_list'"
print(query_update)

manager.update(query_update)

manager.commit()


In [ ]:

# 읽은 데이터 csv파일로 생성. --> 추후 flow : raw data --> python 전처리 --> database --> spotfire 연동 --> spotfire report로 생각중

rst = manager.search("select * from loss_list")
df = pd.DataFrame(rst)
df.to_csv('output/databases.csv',index= False, encoding= 'utf-8-sig')
print(df)
print('done')

===================================================[PC]===========================================
1. 제어판 - 관리 도구 - ODBC 데이터 원본(64비트) - 추가 - MYSQL ODBC 8.0 ANSI Driver 클릭

2. Name / Description 자율  ,     User, Password, Database = 유저이름, DB비번, DB이름  설정 후 완료
===================================================================================================
![dlal.png](./data/PLC_자기유지.PNG)

==============================================[Spotfire]===========================================

1. Data - Add Data - Other - Load Data With ODBC data provider - Odbc Provider 클릭

2. System data source 에서 [PC]단에서 설정한 이름 클릭, 유저이름, DB비번 입력 후 접속

3. 필요한 데이터 베이스에서 데이터 추가 - 데이터형식 이름 작성 - Import 후 완료


예약번호	예약번호 품목	저장 BIN	자재코드	자재명	출고구분
49452229	1	L-1-1-3	BESEN0337	SENSOR,OMRON,E3T-FT11	정품
49455035	1	N-2-4-3	BMSVA0217	SOLENOID VALVE,SMC,SY3240R-5LZ-01	정품
10	10	N-2-3-1	BMCHK0083	SLIDE TABLE,MXQ16-75,SMC	부외품
49455030	1	H-9-3-4	BESEN0658	SENSOR,SMC,ISE30A-01-N-L	정품
49454806	1	O-4-1-2	BEBAT0033	BATTERY,APC,RBC110	정품
10	10	L-3-1-4	BMMOT0991	MOTOR,SPG,S9I90GBH-V12	부외품
49454790	1	N-2-4-3	BMVAL0131	SOL VALVE,SY3240-5LZ,SMC	정품
49452233	1	D-3-4-4	BMCHK1632	CYLINDER,FUJIKURA,SCS-40-48-S1-B0	정품
10	10	E-2-6-1	3359245	PLC MODULE,MITSUBISHI,AJ65SBTB1-32DT3	부외품
49454787	1	F-0-3-3	BEPOW0044	POWER SUPPLY,OMRON,S8JX-N60024C	정품
49454741	1	H-2-1-2	BEPAN0025	BUZZER,PATLITE,BK-24A-J	정품
49452240	1	H-9-4-4	BESEN0826	SENSOR,AUTONICS,TZ4ST-24C	정품
49452239	1	H-2-2-3	BMBLT0523	BELT,MITSUBOSHI,1000-S3M-681G(폭100MM)	정품
49452239	2	I-0-3-3	BMBLT0517	BELT,MBL,S3M681(W:98MM)	정품
49460084	1	D-4-6-1	BMCHK0317	CYLINDER,SMC,MHZ2-16D	정품
49455096	1	O-4-1-2	BEBAT0033	BATTERY,APC,RBC110	정품


In [ ]:
from datetime import datetime

now = datetime.now()

예약번호	예약번호 품목	저장 BIN	자재코드	자재명	출고구분
49452229	1	L-1-1-3	BESEN0337	SENSOR,OMRON,E3T-FT11	정품
49455035	1	N-2-4-3	BMSVA0217	SOLENOID VALVE,SMC,SY3240R-5LZ-01	정품
10	10	N-2-3-1	BMCHK0083	SLIDE TABLE,MXQ16-75,SMC	부외품
49455030	1	H-9-3-4	BESEN0658	SENSOR,SMC,ISE30A-01-N-L	정품
49454806	1	O-4-1-2	BEBAT0033	BATTERY,APC,RBC110	정품
10	10	L-3-1-4	BMMOT0991	MOTOR,SPG,S9I90GBH-V12	부외품
49454790	1	N-2-4-3	BMVAL0131	SOL VALVE,SY3240-5LZ,SMC	정품
49452233	1	D-3-4-4	BMCHK1632	CYLINDER,FUJIKURA,SCS-40-48-S1-B0	정품
10	10	E-2-6-1	3359245	PLC MODULE,MITSUBISHI,AJ65SBTB1-32DT3	부외품
49454787	1	F-0-3-3	BEPOW0044	POWER SUPPLY,OMRON,S8JX-N60024C	정품
49454741	1	H-2-1-2	BEPAN0025	BUZZER,PATLITE,BK-24A-J	정품
49452240	1	H-9-4-4	BESEN0826	SENSOR,AUTONICS,TZ4ST-24C	정품
49452239	1	H-2-2-3	BMBLT0523	BELT,MITSUBOSHI,1000-S3M-681G(폭100MM)	정품
49452239	2	I-0-3-3	BMBLT0517	BELT,MBL,S3M681(W:98MM)	정품
49460084	1	D-4-6-1	BMCHK0317	CYLINDER,SMC,MHZ2-16D	정품
49455096	1	O-4-1-2	BEBAT0033	BATTERY,APC,RBC110	정품